In [1]:
import asyncio
import aiohttp
import pandas as pd
from lxml import etree

In [2]:
all_df = pd.DataFrame()
async def get_page(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
                page =await resp.text()
                page_tree =etree.HTML(page)
                brand_tr_list = page_tree.xpath('//*[@id="compositor-material"]/table/tbody/tr')
                for brand in brand_tr_list:
                    brand_url.append(brand.xpath('./td[1]/a/@href')[0])
                    country.append(brand.xpath('./td[2]')[0].text)
                    print(1)
                    temp_df = await prase_detail(brand_url,country)
                    all_df = pd.concat([all_df,temp_df])

In [3]:
# session = aiohttp.ClientSession()

In [5]:
tasks = []
brand_initials = ['A']
for char in brand_initials:
    print('首字母'+char+'品牌正在爬取中。。。')
    url = 'https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr='+char+'&country_id=&region_id=&wbRanking='
    c = get_page(url)
    task = asyncio.ensure_future(c)
    tasks.append(task)
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

首字母A品牌正在爬取中。。。


RuntimeError: This event loop is already running

In [5]:
url = 'https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr=A&country_id=&region_id=&wbRanking='
async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            page =await resp.text()
            page_tree =etree.HTML(page)
            brand_tr_list = page_tree.xpath('//*[@id="compositor-material"]/table/tbody/tr')
            for brand in brand_tr_list:
                brand_url.append(brand.xpath('./td[1]/a/@href')[0])
                country.append(brand.xpath('./td[2]')[0].text)

In [6]:
async def prase_detail(url,country):
        name_list=[]
        stated_age_list =[]
        strength_list =[]
        size_list =[]
        bottled_list=[]
        casknumber_list=[]
        rating_list=[]
        version_list=[]
        shopping_link_list=[]
        async with session.get(url) as resp:
            page =await resp.text()
            page_tree =etree.HTML(page)
            brand = tree.xpath('//*[@id="company-name"]/h1')[0].text
            tr_list = tree.xpath("//*[@id='compositor-material']/table/tbody/tr")
            for tr in tr_list:
                if tr.xpath("./td[3]/a"):
                    title = tr.xpath("./td[3]/a")[0].text
                    name_list.append(title)

                    stated_age = tr.xpath("./td[4]")[0].text
                    stated_age_list.append(stated_age)

                    strength = tr.xpath("./td[5]")[0].text
                    strength_list.append(strength)

                    size = tr.xpath("./td[6]")[0].text
                    size_list.append(size)

                    bottled= tr.xpath("./td[7]")[0].text
                    bottled_list.append(bottled)

                    casknumber = tr.xpath("./td[8]")[0].text
                    casknumber_list.append(casknumber)

                    rating = tr.xpath("./td[9]")[0].text
                    rating_list.append(rating)

                    version = tr.xpath("./td[10]")[0].text
                    version_list.append(version)

                    shopping_link = tr.xpath("./td[3]//a/@href")[0]
                    shopping_link_list.append(shopping_link)
            dic={'name':name_list,
                 'stated_age':stated_age_list,
                'size':size_list,
                'bottled':bottled_list,
                'casknumber0':casknumber_list,
                'rating':rating_list,
                'version':version_list,
                'shopping_link':shopping_link_list}
            detail_df = pd.DataFrame(dic)
            detail_df['brand']= brand
            detail_df['country'] = country
            return detail_df

In [25]:
table=[]
async def fetch(session, url):
    async with session.get(url) as response:
           return  await response.text

async def parser1(html):
    page_tree =etree.HTML(page)
    brand_tr_list = page_tree.xpath('//*[@id="compositor-material"]/table/tbody/tr')
    for brand in brand_tr_list:
        brand_url= brand.xpath('./td[1]/a/@href')[0]
        country = brand.xpath('./td[2]')[0]
        # 将每本畅销书的上述信息加入到table中
        table.append([brand_url,country])
        
async def download(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        await parser1(html)



In [27]:
brand_initials = ['A','B','C']
urls = []
for char in brand_initials:
    print('首字母'+char+'品牌正在爬取中。。。')
    urls.append('https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr='+char+'&country_id=&region_id=&wbRanking=')
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download(url)) for url in urls]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("object method can't be used in 'await' expression")>
Traceback (most recent call last):
  File "<ipython-input-25-87c6c9dd7899>", line 17, in download
    html = await fetch(session, url)
  File "<ipython-input-25-87c6c9dd7899>", line 4, in fetch
    return  await response.text
TypeError: object method can't be used in 'await' expression


首字母A品牌正在爬取中。。。
首字母B品牌正在爬取中。。。
首字母C品牌正在爬取中。。。


RuntimeError: This event loop is already running

In [23]:
table

[]

In [24]:
urls

['https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr=A&country_id=&region_id=&wbRanking=',
 'https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr=B&country_id=&region_id=&wbRanking=',
 'https://www.whiskybase.com/whiskies/brands?style=table&search=null&chr=C&country_id=&region_id=&wbRanking=']

In [28]:
'''
异步方式爬取当当畅销书的图书信息
'''

import time
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup

# table表格用于储存书本信息
table = []

# 获取网页（文本信息）
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text(encoding='gb18030')

# 解析网页
async def parser(html):
    
    # 利用BeautifulSoup将获取到的文本解析成HTML
    soup = BeautifulSoup(html, "lxml")
    # 获取网页中的畅销书信息
    book_list = soup.find('ul', class_="bang_list clearfix bang_list_mode")('li')

    for book in book_list:
        
        info = book.find_all('div')

        # 获取每本畅销书的排名，名称，评论数，作者，出版社
        rank = info[0].text[0:-1]
        name = info[2].text
        comments = info[3].text.split('条')[0]
        author = info[4].text
        date_and_publisher = info[5].text.split()
        publisher = date_and_publisher[1] if len(date_and_publisher) >=2 else ''

        # 将每本畅销书的上述信息加入到table中
        table.append([rank,name,comments,author,publisher])
        
# 处理网页    
async def download(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        await parser(html)

# 全部网页
urls = ['http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-recent7-0-0-1-%d'%i for i in range(1,26)]

# 统计该爬虫的消耗时间
print('#' * 50)
t1 = time.time() # 开始时间

# 利用asyncio模块进行异步IO处理
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download(url)) for url in urls]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

# 将table转化为pandas中的DataFrame并保存为CSV格式的文件
df = pd.DataFrame(table, columns=['rank','name','comments','author','publisher'])
df.to_csv('E://douban/dangdang.csv',index=False)
    
t2 = time.time() # 结束时间
print('使用aiohttp，总共耗时：%s' % (t2 - t1))
print('#' * 50)

##################################################


RuntimeError: This event loop is already running

_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("object method can't be used in 'await' expression")>
Traceback (most recent call last):
  File "<ipython-input-11-0fbad080eaef>", line 17, in download
    html = await fetch(session, url)
  File "<ipython-input-11-0fbad080eaef>", line 4, in fetch
    return await response.text
TypeError: object method can't be used in 'await' expression
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("object method can't be used in 'await' expression")>
Traceback (most recent call last):
  File "<ipython-input-17-343554ffb92d>", line 17, in download
    html = await fetch(session, url)
  File "<ipython-input-17-343554ffb92d>", line 4, in fetch
    await response.text
TypeError: object method can't be used in 'await' expression
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("object method can't 

In [5]:
l =[0,1,2,3,4,5,6,7]
j = 0
for i in l [3]:
    print(i)

0
1
2


In [6]:
l[3]

3